In [ ]:
#instructions to download miniconda3 from the terminal
#mkdir -p ~/miniconda3
#wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
#bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
#rm ~/miniconda3/miniconda.sh

#clone repo
#source ~/miniconda3/bin/activate
#conda init --all
#source ~/.bashrc

#conda environment
#clone the repository
#cd ~
#git clone https://github.com/hakyimlab/MetaXcan.git

#conda environment for MetaXcan
#cd MetaXcan/software
#conda env create -f conda_env.yaml
#conda activate imlabtools

In [ ]:
#install.packages('R.utils')
library(data.table)
library(dplyr)
library(tidyr)

In [ ]:
#download reference genome
#system("wget https://ftp.ncbi.nlm.nih.gov/snp/organisms/human_9606_b151_GRCh38p7/VCF/All_20180418.vcf.gz")

#save for future use
#system("gsutil cp All_20180418.vcf.gz gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data/")

#create file of chr and pos columns only to use for filtering
#system("gsutil cat gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data/filtered_261.2.tsv | awk 'NR > 1 {print $8, $9}' > subset_261.2.tsv")

#remove chr prefix
#system("sed -e 's/chr//' -e 's/^X /23 /' subset_261.2.tsv > nochr261.2.tsv")

#filter large file, eliminating SNPs not present in sumstats file
#system("awk 'NR==FNR {a[$1,$2]; next} !/^#/ && ($1,$2) in a' nochr261.2.tsv <(zcat All_20180418.vcf.gz) > filtered_20180418.vcf")

#remove metadata rows
#system("awk '!/^##/' filtered_20180418.vcf > 261.2ref.vcf")

#copy to bucket
#system("gsutil cp 261.2ref.vcf gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data/")

In [ ]:
#read in table
name_of_file <- 'filtered_261.2.tsv'

my_bucket <- Sys.getenv('WORKSPACE_BUCKET')
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file, " ."), intern=T)
system("gsutil cp gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data/filtered_261.2.tsv")

table <- fread(name_of_file, header=TRUE)

#check table
#head(table)

In [ ]:
#reformat locus column to chr_pos_ref_alt_b38
table$locus_formatted <- gsub(":", "_", table$locus) #colon to underscore
table$alleles_formatted <- gsub('\\["', "", table$alleles)  #remove opening [
table$alleles_formatted <- gsub('"\\]', "", table$alleles_formatted)  #remove closing ]
table$alleles_formatted <- gsub('","', "_", table$alleles_formatted)  #comma to underscore

#split allele column
table <- table %>%
  separate(alleles_formatted, into = c("A1", "A2"), sep = "_", remove=F)

#combine strings
table$SNP <- paste0(table$locus_formatted, "_", table$alleles_formatted, "_b38")

#remove intermediate columns
table$locus_formatted <- NULL
table$alleles_formatted <- NULL

#edit sex chromosomes
table$CHR <- gsub("X", "23", table$CHR)
table$CHR <- gsub("Y", "24", table$CHR)

#check table
#head(table)

In [ ]:
#read in
name_of_vcf <- '261.2ref.vcf'
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_vcf, " ."), intern=T)
system("gsutil cp gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data/261.2ref.vcf")

reference_data <- fread(name_of_vcf, header = FALSE, sep='\t')
reference_data <- reference_data[,1:3]
colnames(reference_data) <- c("CHR", "POS", "rsID")

#format data for matching
table$CHR <- as.character(table$CHR)
table$POS <- as.character(table$POS)

reference_data$CHR <- paste0("chr", reference_data$CHR)
reference_data$CHR <- as.character(reference_data$CHR)
reference_data$POS <- as.character(reference_data$POS)

#check tables
#head(reference_data)
#head(table)

In [ ]:
#merge files
merged_data <- merge(table, reference_data[, c("CHR", "POS", "rsID")], by = c("CHR", "POS"), all.x = TRUE)

#remove un-needed columns
filtered_merged_data <- merged_data[, c(1, 2, 13, 14, 15, 16, 5, 6, 8)]

#check table
#head(filtered_merged_data)

In [ ]:
#final formatting

#format chromosomes
filtered_merged_data$CHR <- gsub("chr", "", filtered_merged_data$CHR)
filtered_merged_data$CHR <- gsub("X", "23", filtered_merged_data$CHR)
filtered_merged_data$CHR <- gsub("Y", "24", filtered_merged_data$CHR)

#make numeric
filtered_merged_data$CHR <- as.numeric(filtered_merged_data$CHR)
filtered_merged_data$POS <- as.numeric(filtered_merged_data$POS)

#column names
filtered_merged_data$P <- filtered_merged_data$Pvalue
filtered_merged_data$Pvalue <- NULL

#sort by chr, pos
filtered_merged_data <- filtered_merged_data %>%
  arrange(CHR, POS)

#check table
#print(filtered_merged_data)

In [ ]:
#write table
destination_filename <- 'formatted_261.2.tsv'

#store the dataframe in current workspace
write.table(filtered_merged_data, destination_filename, col.names=TRUE, row.names=FALSE, quote=FALSE, sep="\t")

#copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

In [ ]:
#check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/"), intern=T)

In [ ]:
#run S-PrediXcan
#os.chdir("/home/jupyter/packages/MetaXcan/software")
#os.system("conda activate imlabtools")

#os.system("SPrediXcan.py --model_db_path allofus_test/eqtl/mashr/mashr_Stomach.db --covariance allofus/test/eqtl/mashr/mashr_Stomach.txt.gz --gwas_file gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data/formatted_261.2.tsv --snp_column SNP --effect_allele_column A1 --non_effect_allele_column A2 --beta_column BETA --se_column SE --pvalue_column Pvalue --output_file results/261.2.csv")

#save to bucket
#reference_filename <- 'ref_build38.vcf'

#store the dataframe in current workspace
#write.table(reference_data, reference_filename, col.names=TRUE, row.names=FALSE, quote=FALSE, sep="\t")

#copy the file from current workspace to the bucket
#system(paste0("gsutil cp ./", reference_filename, " ", my_bucket, "/data/"), intern=T)